In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session #sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy import Column, Integer, String

import pandas as pd
import numpy as np

In [2]:
engine = create_engine('sqlite:///../data/football.db')

Base = automap_base()
Base.prepare(engine, reflect=True)

cfb_positions = Base.classes.cfb_positions
cfb_recruits = Base.classes.cfb_recruits
nfl_draft = Base.classes.nfl_draft
nfl_positions = Base.classes.nfl_positions
nfl_teams = Base.classes.nfl_teams

session = Session(engine)
inspector = inspect(engine)

In [4]:
recruits = pd.read_sql_table('cfb_recruits', engine)
draftees = pd.read_sql_table('nfl_draft', engine)

In [19]:
draftees.head(1)

,PRIMARY_KEY,YEAR,RND,PICK,FIRST_NAME,LAST_NAME,TM,POS,SCHOOL
0,1,2009,1,1,Matthew,Stafford,DET,QB,Georgia


In [20]:
results = session.query(cfb_recruits, nfl_draft).filter(cfb_recruits.FIRST_NAME == nfl_draft.FIRST_NAME).filter(cfb_recruits.LAST_NAME == nfl_draft.LAST_NAME)

In [29]:
draftees['PLAYER'] = draftees['FIRST_NAME'] + ' ' + draftees['LAST_NAME']
draftees.head()

,PRIMARY_KEY,YEAR,RND,PICK,FIRST_NAME,LAST_NAME,TM,POS,SCHOOL,PLAYER
0,1,2009,1,1,Matthew,Stafford,DET,QB,Georgia,Matthew Stafford
1,2,2009,1,2,Jason,Smith,STL,T,Baylor,Jason Smith
2,3,2009,1,3,Tyson,Jackson,KAN,DE,LSU,Tyson Jackson
3,4,2009,1,4,Aaron,Curry,SEA,LB,Wake Forest,Aaron Curry
4,5,2009,1,5,Mark,Sanchez,NYJ,QB,USC,Mark Sanchez


In [30]:
recruits.head()

,PRIMARY_KEY,PLAYER,FIRST_NAME,LAST_NAME,POS,YEAR,RNK,STAR,247_SCORE,COLLEGE,CITY,STATE,LAT,LNG,HIGH_SCHOOL
0,1,Joe McKnight,Joe,McKnight,RB,2007,1,5,0.9997,USC,New Orleans,LA,30.032802,-89.975005,John Curtis
1,2,Lionel Breaux,Lionel,Breaux,WR,2007,305,3,0.8795,Ole Miss,New Orleans,LA,30.032802,-89.975005,McDonogh 35
2,3,Korey Williams,Korey,Williams,ILB,2007,519,3,0.8559,Southern Miss,New Orleans,LA,30.032802,-89.975005,Edna Karr
3,4,Tyler Stradford,Tyler,Stradford,WR,2007,523,3,0.8559,Oklahoma,New Orleans,LA,30.032802,-89.975005,O Perry Walker
4,5,Delvin Breaux,Delvin,Breaux,CB,2007,665,3,0.8479,LSU,New Orleans,LA,30.032802,-89.975005,McDonogh 35


In [35]:
drafted = pd.merge(recruits, draftees, on='PLAYER')
drafted.head()

,PRIMARY_KEY_x,PLAYER,FIRST_NAME_x,LAST_NAME_x,POS_x,YEAR_x,RNK,STAR,247_SCORE,COLLEGE,...,HIGH_SCHOOL,PRIMARY_KEY_y,YEAR_y,RND,PICK,FIRST_NAME_y,LAST_NAME_y,TM,POS_y,SCHOOL
0,1,Joe McKnight,Joe,McKnight,RB,2007,1,5,0.9997,USC,...,John Curtis,369,2010,4,112,Joe,McKnight,NYJ,RB,USC
1,35,Tyrann Mathieu,Tyrann,Mathieu,CB,2010,223,4,0.9132,LSU,...,St. Augustine,1092,2013,3,69,Tyrann,Mathieu,ARI,DB,LSU
2,51,Trai Turner,Trai,Turner,OG,2011,355,3,0.8765,LSU,...,St. Augustine,1369,2014,3,92,Trai,Turner,CAR,OL,LSU
3,67,Deion Jones,Deion,Jones,OLB,2012,378,3,0.8899,LSU,...,Jesuit,1842,2016,2,52,Deion,Jones,ATL,OLB,LSU
4,72,Lorenzo Doss,Lorenzo,Doss,ATH,2012,1123,3,0.8354,Tulane,...,St. Augustine,1697,2015,5,164,Lorenzo,Doss,DEN,CB,Tulane


In [36]:
drafted.groupby('STAR').count()

,PRIMARY_KEY_x,PLAYER,FIRST_NAME_x,LAST_NAME_x,POS_x,YEAR_x,RNK,247_SCORE,COLLEGE,CITY,...,HIGH_SCHOOL,PRIMARY_KEY_y,YEAR_y,RND,PICK,FIRST_NAME_y,LAST_NAME_y,TM,POS_y,SCHOOL
STAR,,,,,,,,,,,,,,,,,,,,,
2,216,216,216,216,216,216,216,216,216,216,...,216,216,216,216,216,216,216,216,216,216
3,621,621,621,621,621,621,621,621,621,621,...,621,621,621,621,621,621,621,621,621,621
4,386,386,386,386,386,386,386,386,386,386,...,386,386,386,386,386,386,386,386,386,386
5,108,108,108,108,108,108,108,108,108,108,...,108,108,108,108,108,108,108,108,108,108


In [33]:
recruits.groupby('STAR').count()

,PRIMARY_KEY,PLAYER,FIRST_NAME,LAST_NAME,POS,YEAR,RNK,247_SCORE,COLLEGE,CITY,STATE,LAT,LNG,HIGH_SCHOOL
STAR,,,,,,,,,,,,,,
1,27,27,27,27,27,27,27,27,27,27,27,27,27,27
2,5278,5278,5278,5278,5278,5278,5278,5278,5278,5278,5278,5278,5278,5278
3,7675,7675,7675,7675,7675,7675,7675,7675,7675,7675,7675,7675,7675,7675
4,1722,1722,1722,1722,1722,1722,1722,1722,1722,1722,1722,1722,1722,1722
5,190,190,190,190,190,190,190,190,190,190,190,190,190,190


In [46]:
drafted.groupby(['SCHOOL']).count().sort_values('PLAYER', ascending=False)

,PRIMARY_KEY_x,PLAYER,FIRST_NAME_x,LAST_NAME_x,POS_x,YEAR_x,RNK,STAR,247_SCORE,COLLEGE,...,LNG,HIGH_SCHOOL,PRIMARY_KEY_y,YEAR_y,RND,PICK,FIRST_NAME_y,LAST_NAME_y,TM,POS_y
SCHOOL,,,,,,,,,,,,,,,,,,,,,
Alabama,46,46,46,46,46,46,46,46,46,46,...,46,46,46,46,46,46,46,46,46,46
Clemson,38,38,38,38,38,38,38,38,38,38,...,38,38,38,38,38,38,38,38,38,38
LSU,37,37,37,37,37,37,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37
Florida St.,34,34,34,34,34,34,34,34,34,34,...,34,34,34,34,34,34,34,34,34,34
Georgia,33,33,33,33,33,33,33,33,33,33,...,33,33,33,33,33,33,33,33,33,33
Ohio St.,30,30,30,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30
Florida,30,30,30,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30
USC,29,29,29,29,29,29,29,29,29,29,...,29,29,29,29,29,29,29,29,29,29
Notre Dame,28,28,28,28,28,28,28,28,28,28,...,28,28,28,28,28,28,28,28,28,28


In [52]:
five_drafted = drafted.loc[drafted['STAR'] == 5]
five_drafted.groupby('SCHOOL').count().sort_values('PLAYER',ascending=False).head(10)

,PRIMARY_KEY_x,PLAYER,FIRST_NAME_x,LAST_NAME_x,POS_x,YEAR_x,RNK,STAR,247_SCORE,COLLEGE,...,LNG,HIGH_SCHOOL,PRIMARY_KEY_y,YEAR_y,RND,PICK,FIRST_NAME_y,LAST_NAME_y,TM,POS_y
SCHOOL,,,,,,,,,,,,,,,,,,,,,
Alabama,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
Florida,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
USC,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
Florida St.,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
Georgia,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
Clemson,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
Miami (FL),4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
Ohio St.,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
Texas,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [54]:
five_recruited = recruits.loc[recruits['STAR'] == 5]
five_recruited.groupby('COLLEGE').count().sort_values('PLAYER', ascending=False).head(10)

,PRIMARY_KEY,PLAYER,FIRST_NAME,LAST_NAME,POS,YEAR,RNK,STAR,247_SCORE,CITY,STATE,LAT,LNG,HIGH_SCHOOL
COLLEGE,,,,,,,,,,,,,,
Florida,21,21,21,21,21,21,21,21,21,21,21,21,21,21
USC,18,18,18,18,18,18,18,18,18,18,18,18,18,18
Florida State,17,17,17,17,17,17,17,17,17,17,17,17,17,17
Alabama,15,15,15,15,15,15,15,15,15,15,15,15,15,15
Texas,14,14,14,14,14,14,14,14,14,14,14,14,14,14
Georgia,13,13,13,13,13,13,13,13,13,13,13,13,13,13
Ohio State,10,10,10,10,10,10,10,10,10,10,10,10,10,10
LSU,9,9,9,9,9,9,9,9,9,9,9,9,9,9
Notre Dame,8,8,8,8,8,8,8,8,8,8,8,8,8,8


In [53]:
four_drafted = drafted.loc[drafted['STAR'] == 4]
four_drafted.groupby('SCHOOL').count().sort_values('PLAYER',ascending=False).head(10)

,PRIMARY_KEY_x,PLAYER,FIRST_NAME_x,LAST_NAME_x,POS_x,YEAR_x,RNK,STAR,247_SCORE,COLLEGE,...,LNG,HIGH_SCHOOL,PRIMARY_KEY_y,YEAR_y,RND,PICK,FIRST_NAME_y,LAST_NAME_y,TM,POS_y
SCHOOL,,,,,,,,,,,,,,,,,,,,,
Alabama,25,25,25,25,25,25,25,25,25,25,...,25,25,25,25,25,25,25,25,25,25
LSU,21,21,21,21,21,21,21,21,21,21,...,21,21,21,21,21,21,21,21,21,21
Notre Dame,18,18,18,18,18,18,18,18,18,18,...,18,18,18,18,18,18,18,18,18,18
USC,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
Oklahoma,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
Georgia,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
Clemson,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
Florida,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,14,14,14,14,14,14
Ohio St.,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,14,14,14,14,14,14


In [55]:
four_recruited = recruits.loc[recruits['STAR'] == 4]
four_recruited.groupby('COLLEGE').count().sort_values('PLAYER', ascending=False).head(10)

,PRIMARY_KEY,PLAYER,FIRST_NAME,LAST_NAME,POS,YEAR,RNK,STAR,247_SCORE,CITY,STATE,LAT,LNG,HIGH_SCHOOL
COLLEGE,,,,,,,,,,,,,,
Texas,88,88,88,88,88,88,88,88,88,88,88,88,88,88
Alabama,79,79,79,79,79,79,79,79,79,79,79,79,79,79
LSU,71,71,71,71,71,71,71,71,71,71,71,71,71,71
Florida,68,68,68,68,68,68,68,68,68,68,68,68,68,68
USC,65,65,65,65,65,65,65,65,65,65,65,65,65,65
Notre Dame,63,63,63,63,63,63,63,63,63,63,63,63,63,63
Ohio State,63,63,63,63,63,63,63,63,63,63,63,63,63,63
Michigan,60,60,60,60,60,60,60,60,60,60,60,60,60,60
Oklahoma,60,60,60,60,60,60,60,60,60,60,60,60,60,60


In [82]:
test = drafted['STAR'].groupby(drafted['SCHOOL']).value_counts()
df_test = test.to_frame()
df_test = df_test.rename(columns={'STAR':'star_drafted'})
df_test = df_test.reset_index()
df_test.head(10)

,SCHOOL,STAR,star_drafted
0,,2,1
1,,3,1
2,Akron,2,1
3,Ala-Birmingham,2,1
4,Alabama,4,25
5,Alabama,5,10
6,Alabama,3,9
7,Alabama,2,2
8,Arizona,3,7
9,Arizona,4,2


In [91]:
df_drafted_stars = df_test.replace(to_replace='West. Kentucky', value='Western Kentucky')
df_drafted_stars

,SCHOOL,STAR,star_drafted
0,,2,1
1,,3,1
2,Akron,2,1
3,Ala-Birmingham,2,1
4,Alabama,4,25
5,Alabama,5,10
6,Alabama,3,9
7,Alabama,2,2
8,Arizona,3,7
9,Arizona,4,2


In [93]:
school_stars = recruits['STAR'].groupby(recruits['COLLEGE']).value_counts()
df_school_stars = school_stars.to_frame()
df_school_stars = df_school_stars.rename(columns={'STAR':'star_recruited'})
df_school_stars = df_school_stars.reset_index()
df_school_stars

,COLLEGE,STAR,star_recruited
0,Air Force,2,79
1,Air Force,3,14
2,Air Force,1,2
3,Akron,2,88
4,Akron,3,29
5,Alabama,4,79
6,Alabama,3,50
7,Alabama,5,15
8,Alabama,2,1
9,Alabama State,2,1


In [92]:
star_merge = pd.merge(df_drafted_stars, df_school_stars, how='left', left_on=['SCHOOL','STAR'], right_on=['COLLEGE','STAR'])
star_merge

,SCHOOL,STAR,star_drafted,COLLEGE,star_recruited
0,,2,1,NaN,NaN
1,,3,1,NaN,NaN
2,Akron,2,1,Akron,88.0
3,Ala-Birmingham,2,1,NaN,NaN
4,Alabama,4,25,Alabama,79.0
5,Alabama,5,10,Alabama,15.0
6,Alabama,3,9,Alabama,50.0
7,Alabama,2,2,Alabama,1.0
8,Arizona,3,7,Arizona,82.0
9,Arizona,4,2,Arizona,10.0
